## Obstacle Avoidance using Collision Cone

* A single holonomic robot is supposed to reach its goal by avoiding collision with moving obstacles. We use velocity obstacle formulation to perform goal reaching obstacle avoidance in a dynamic environment.
* We try to calculate an optimal velocity such that the agent reaches the goal while avoiding the obstacles using collision cone constraints. 
* We perform a sampling based method to generate different velocities within a range and then find the optimal velocity with minimum cost.

<!-- ![VO.png](./VO.pngwidth = 400) -->
<div>
<img src="./VO.png" width="300"/>
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

### Assigning the goal positions of robot and obstacles
* We fix the goal positions initially and set a timestep of 0.01

In [2]:
dt = 0.01

goal_robot = np.array([-5,15])
goal_obstacle = np.array([10,0]) 
goal_obstacle1 = np.array([0,0]) 
goal_obstacle2 = np.array([-10,10]) 

### Classes for Robot and Moving Obstacle

In [3]:
class Robot(object):
    def __init__(self, robot_radius, rx, ry, vx, vy):
        self.rx = rx
        self.ry = ry
        self.vx = vx
        self.vy = vy
        self.radius = robot_radius 
        self.velocity = vx,vy
        self.position = rx,ry

    def new_pos_x(self, dt):
        return self.rx + self.vx*dt

    def new_pos_y(self, dt):
        return self.ry + self.vy*dt

    def update_velocity(self, velocity):
        self.vx = velocity[0]
        self.vy = velocity[1]
        self.velocity = velocity

    def update_position(self,position):
        self.rx = position[0]
        self.ry = position[1]
        self.position = position

In [4]:
class Obstacle(object):
    def __init__(self, obs_radius, rx, ry, vx, vy):
        self.radius = obs_radius 
        self.rx = rx
        self.ry = ry
        self.vx = vx
        self.vy = vy
        self.velocity = vx,vy
        self.position = rx,ry


    def new_pos_x(self, dt):
        return self.rx + self.vx*dt

    def new_pos_y(self, dt):
        return self.ry + self.vy*dt

    def update_velocity(self, velocity):
        self.vx = velocity[0]
        self.vy = velocity[1]
        self.velocity = velocity


    def update_position(self, position):
        self.rx=position[0]
        self.ry=position[1]
        self.position = position

### Checking the collision cone constraint
* The following expression represents the collision cone constraint.

$$  r^{2} - \frac{||r.(v_{rb}+ \Delta v_{r})||^{2}}{||v_{rb}+\Delta v_{r}||^{2}} \geq R^{2}$$

    
 where 
    $r$ - position vector of robot, $v_{r}$ - velocity of robot, $v_{rb}$ - relative velocity of robot wrt obstacle

      
$$(v_{rb}+ \Delta v_{r}) = v_{rnew} - v_{b} $$
* The velocities with non-negative constraint value are selected and the one corresponding to least cost is chosen finally. The constraint_val is returned by collision_cone_val() function

$$ ({r^{2} - R^{2})||v_{rb}+\Delta v_{r}||^{2} - ||r.(v_{rb}+ \Delta v_{r})||^{2}} \geq  0 $$ 


In [5]:
def collision_cone_val(robot, obstacle):
    rx = robot.rx
    ry = robot.ry
    vrx = robot.vx
    vry = robot.vy

    obx = obstacle.rx
    oby = obstacle.ry
    vobx = obstacle.vx
    voby = obstacle.vy
    R = robot.radius + obstacle.radius
    # if constraint_val >= 0, no collision , else there will be a collision in the future
    constraint_val = -((rx - obx) * (vrx - vobx) + (ry - oby) * (vry - voby)) ** 2 + ( -R ** 2 + ( rx- obx) ** 2 + (ry - oby) ** 2) * ((vrx - vobx)**2 + (vry - voby)**2)
    return constraint_val

def Collision_check(robot, obstacle):
    collision = np.linalg.norm(obstacle.position - robot.position) < (robot.radius + obstacle.radius)
    if collision:
        return True
    return False


### Sampling velocities

In [6]:
def Sample_velocities():
    velocity = np.arange(-5.0,5.0,1)
    v_samples = [[x,y] for x in velocity for y in velocity]
    return np.array(v_samples)


### Finding the optimal velocity for no collision
* We find the costs by L2 norm between the sampled velocities and the desired robot velocities.
* There are 3 colllision cones corresponding to the 3 obstacles and we choose the indices that satisfy the collision constraints. For these indices, we take find the costs and take the common costs. 
* From the commmon costs , we return the optimal velocity that gives the minimum cost.

In [7]:
def Find_Optimalvelocity(desired_robot_velocity, sampled_velocities, cone_vals, cone_vals1, cone_vals2):

    costs = []
    for vel in sampled_velocities:
        costs.append(np.linalg.norm(desired_robot_velocity - vel))
    costs = np.array(costs)

    nocollide_index = np.where(cone_vals >= 0)[0]

    if len(nocollide_index) != 0:
        possible_costs = np.take(costs, nocollide_index)
    else:
        possible_costs = costs

    nocollide_index1 = np.where(cone_vals1 >= 0)[0]
    if len(nocollide_index1) != 0:
        possible_costs1 = np.take(costs,nocollide_index1)
    else:
        possible_costs1 = costs

    nocollide_index2 = np.where(cone_vals2 >= 0)[0]
    if len(nocollide_index2) != 0:
        possible_costs2 = np.take(costs,nocollide_index2)
    else:
        possible_costs2 = costs

    common_costs = [value for value in possible_costs if (value in possible_costs1 and value in possible_costs2)]
    optimal_cost = np.amin(common_costs)

    optimal_velocity_index = np.where(costs == optimal_cost)[0]
    optimal_velocity = sampled_velocities[optimal_velocity_index[0]]

    return optimal_velocity


### Plotting the Optimal Trajectory

In [8]:
def PlotTrajectory(trajectory, robot, obstacle, obstacle1, obstacle2):
    plt.cla()
    plt.gcf().canvas.mpl_connect('key_release_event',lambda event: [exit(0) if event.key == 'escape' else None])
    plt.scatter(goal_robot[0],goal_robot[1],marker="X", color='r')
    plt.scatter(10,0,marker="o", color='darkgreen')

    circle1 = plt.Circle((robot.rx,robot.ry), robot.radius, color='black')
    circle2 = plt.Circle((obstacle.rx,obstacle.ry), obstacle.radius, color='navy')
    circle3 = plt.Circle((obstacle1.rx,obstacle1.ry), obstacle1.radius, color='green')
    circle4 = plt.Circle((obstacle2.rx,obstacle2.ry), obstacle2.radius, color='crimson')

    plt.gcf().gca().add_artist(circle1)
    plt.gcf().gca().add_artist(circle2)
    plt.gcf().gca().add_artist(circle3)
    plt.gcf().gca().add_artist(circle4)

    plot_traj = np.array(trajectory)
    posx, posy = plot_traj.T
    plt.plot(posx, posy,linewidth = "2", color = 'gray')
    plt.xlim(-10, 15)
    plt.ylim(-5, 20)
    plt.title("Velocity Obstacle using Sampling")

    plt.pause(dt)


## The Velocity Obstacle
We first fix the start and goal positions for the robot and obstacles. To find the path from start to goal -
* We update the robot and obstacle velocities with desired velocities
* Then, we sample velocities and search for those that satisfy the collision cone constraints. 
* These velocities are passed to the Find_Optimalvelocity() function to return the velocity with minimum cost.
* The new robot and obstacles positions are found with the optimal velocity. 
* The optimal trajectory is plotted.

The loop terminates when the robot reaches the goal.

In [9]:
def Velocity_Obstacle():
    robot = Robot(1, 10,0,0,0)
    obstacle = Obstacle(1,0,10,0,0) 
    obstacle1 = Obstacle(1,9,9,0,0)
    obstacle2 = Obstacle(1,0,15,0,0)
    
    trajectory = []
    while ((np.linalg.norm(goal_robot - robot.position))>1):
        desired_robot_velocity = ((goal_robot-robot.position)/np.linalg.norm(goal_robot-robot.position))*5
        desired_obstacle_velocity = ((goal_obstacle-obstacle.position)/np.linalg.norm(goal_obstacle-obstacle.position))*1
        desired_obstacle1_velocity = ((goal_obstacle1-obstacle1.position)/np.linalg.norm(goal_obstacle1-obstacle1.position))*5.0754
        desired_obstacle2_velocity = ((goal_obstacle2-obstacle2.position)/np.linalg.norm(goal_obstacle2-obstacle2.position))*0.8

        robot.update_velocity(desired_robot_velocity)
        obstacle.update_velocity(desired_obstacle_velocity) #blue
        obstacle1.update_velocity(desired_obstacle1_velocity) #green
        obstacle2.update_velocity(desired_obstacle2_velocity) #black

        cone_vals = []
        cone_vals1=[]
        cone_vals2=[]

        sampled_velocities = Sample_velocities()
        for vel in sampled_velocities:
            robot.update_velocity(vel)
            cone_vals.append(collision_cone_val(robot, obstacle))
            cone_vals1.append(collision_cone_val(robot, obstacle1))
            cone_vals2.append(collision_cone_val(robot, obstacle2))

        cone_vals = np.array(cone_vals)
        cone_vals1 = np.array(cone_vals1)
        cone_vals2 = np.array(cone_vals2)

        optimal_velocity = Find_Optimalvelocity(desired_robot_velocity, sampled_velocities, cone_vals, cone_vals1, cone_vals2)
        robot.update_velocity(optimal_velocity)

        robot.update_position(np.asarray((robot.new_pos_x(dt),robot.new_pos_y(dt))))
        obstacle.update_position(np.asarray((obstacle.new_pos_x(dt),obstacle.new_pos_y(dt))))
        obstacle1.update_position(np.asarray((obstacle1.new_pos_x(dt),obstacle1.new_pos_y(dt))))
        obstacle2.update_position(np.asarray((obstacle2.new_pos_x(dt),obstacle2.new_pos_y(dt))))

        trajectory.append(robot.position)
        PlotTrajectory(trajectory, robot, obstacle, obstacle1, obstacle2)

    plt.pause(0.0001)
    plt.show()

In [10]:
def Velocity_Obstacle2():
    robot = Robot(1, 10,0,0,0)
    obstacle = Obstacle(2,0,10,0,0) 
    obstacle1 = Obstacle(1.2,9,9,0,0)
    obstacle2 = Obstacle(1.5,0,15,0,0)
    
    trajectory = []
    while ((np.linalg.norm(goal_robot - robot.position))>1):
        desired_robot_velocity = ((goal_robot-robot.position)/np.linalg.norm(goal_robot-robot.position))*5
        desired_obstacle_velocity = ((goal_obstacle-obstacle.position)/np.linalg.norm(goal_obstacle-obstacle.position))*1
        desired_obstacle1_velocity = ((goal_obstacle1-obstacle1.position)/np.linalg.norm(goal_obstacle1-obstacle1.position))*5.0754
        desired_obstacle2_velocity = ((goal_obstacle2-obstacle2.position)/np.linalg.norm(goal_obstacle2-obstacle2.position))*0.8

        robot.update_velocity(desired_robot_velocity)
        obstacle.update_velocity(desired_obstacle_velocity) #blue
        obstacle1.update_velocity(desired_obstacle1_velocity) #green
        obstacle2.update_velocity(desired_obstacle2_velocity) #black

        cone_vals = []
        cone_vals1=[]
        cone_vals2=[]

        sampled_velocities = Sample_velocities()
        for vel in sampled_velocities:
            robot.update_velocity(vel)
            cone_vals.append(collision_cone_val(robot, obstacle))
            cone_vals1.append(collision_cone_val(robot, obstacle1))
            cone_vals2.append(collision_cone_val(robot, obstacle2))

        cone_vals = np.array(cone_vals)
        cone_vals1 = np.array(cone_vals1)
        cone_vals2 = np.array(cone_vals2)

        optimal_velocity = Find_Optimalvelocity(desired_robot_velocity, sampled_velocities, cone_vals, cone_vals1, cone_vals2)
        robot.update_velocity(optimal_velocity)

        robot.update_position(np.asarray((robot.new_pos_x(dt),robot.new_pos_y(dt))))
        obstacle.update_position(np.asarray((obstacle.new_pos_x(dt),obstacle.new_pos_y(dt))))
        obstacle1.update_position(np.asarray((obstacle1.new_pos_x(dt),obstacle1.new_pos_y(dt))))
        obstacle2.update_position(np.asarray((obstacle2.new_pos_x(dt),obstacle2.new_pos_y(dt))))

        trajectory.append(robot.position)
        PlotTrajectory(trajectory, robot, obstacle, obstacle1, obstacle2)

    plt.pause(0.0001)
    plt.show()

In [11]:
Velocity_Obstacle()
Velocity_Obstacle2()